In [367]:
import random
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML

In [368]:
def read(filename):
    data_raw = pd.read_csv(filename, header = None)
    data_raw.drop([1,3,5,6,7,8,9,13],axis = 1, inplace= True)
    data_raw.columns = [0,1,2,3,4,5,6]
    return(data_raw)

In [369]:
def f(x, a, b):
    f = np.dot(a.T, x) + b
    return f

In [370]:
def cost_function(x, y, a, b, lam):
    cost_function = np.maximum(0,(1-np.dot(y,f(x,a,b))))+(1/2)*lam*np.dot(a.T,a)
    return cost_function

In [371]:
def lamda_split(data_raw):
    arr = data_raw.values.reshape(data_raw.iloc[:,0].size, data_raw.columns.size)
    np.random.seed(seed = 1)
    np.random.shuffle(arr)
    n = 4400
    arr1,arr2 = np.split(arr,[n])
    lamda_test = pd.DataFrame(arr1)
    train = pd.DataFrame(arr2)
    return train, lamda_test

In [372]:
def yita_split(train, seed):
    arr = train.values.reshape(train.iloc[:,0].size, train.columns.size)
    np.random.seed(seed = seed)
    np.random.shuffle(arr)
    n = 50
    arr1,arr2 = np.split(arr,[n])
    train_yita = pd.DataFrame(arr2)
    test_yita = pd.DataFrame(arr1)
    return train_yita, test_yita

In [373]:
def getxy(data):
    data_mean = np.zeros(6)
    data_std = np.zeros(6)
    p = data[0].count()
    x = np.zeros((p,6))
    y0 = []
    for i in range(6):
        data_mean[i] = data[i].mean()#data_var[i] = data[i].var()
        data_std[i] = data[i].std()
    for m in range(p):
        for n in range(6): #x[m][n] = ((data[n][m] - data_mean[n])/math.sqrt(data_var[n])
            x[m][n] = ((data[n][m] - data_mean[n])/(data_std[n]))
        y0.append(data[6][m])
    y = np.zeros(len(y0))
    for o in range(len(y0)):
        if y0[o] == ' >50K':
            y[o] = 1
        if y0[o] == ' <=50K':
            y[o] = -1
    return x, y 

In [374]:
def gradient_of_cost(x, y, u, lam):
    if (y*f(x,u[0],u[1])) >= 1:
        gradient_b = 0
        gradient_a = lam * u[0]
    else:
        gradient_b = -y
        gradient_a = -y*x + lam * u[0]
    return [gradient_a, gradient_b]

In [375]:
# Compare real and predicted labels and get accuracy
def accuracy(a,b,x,y):
    right = 0
    for i in range(len(x)):
        s = np.dot(a.T,x[i].T)+b
        s = np.sign(s)
        if y[i] == s:
            right = right +1
        acc = right/y.size
    return acc

In [377]:
def trainSVM(train):
  # initial a and b
    seasons = 60
    u_list = []
    acc_list = []
    season_list = []
    train_lamda,test_lamda = lamda_split(train)
    for lam in [1e-2]:
        for k in range(seasons):
            a_est = np.full((6,1),1)
            a_est = np.matrix(a_est)
            b_est = 3
            new_u = [a_est,b_est]
            seed = np.random.randint(0, 100)
            train_yita, test_yita = yita_split(train_lamda, seed)
            x, y = getxy(train_yita)
            eta = 1/(k+1+50)
            steps = 900
            x = np.matrix(x)
            x1,y1 = getxy(test_yita)
            acc = []
            stepac = []
            for i in range(steps):
                n = np.random.randint(0, len(x)-1) # Batch size of 1.
                g = gradient_of_cost(x[n].T, y[n], new_u, lam)
                new_u[0] = new_u[0] - eta*g[0]
                new_u[1] = new_u[1] - eta*g[1]
                if (i+1)%30 == 0:
                    if accuracy(new_u[0], new_u[1], x1,y1)!=0:
                        acc.append(accuracy(new_u[0], new_u[1], x1,y1))
                        stepac.append((i+1))
            if acc != []:
                print('the result of season', (k+1),'and lamda is:', lam,'eta is:',eta)
                print('Accuracy is:',acc)
                print('Step is:',stepac)
                print(new_u)
                u_list.append(new_u)
                acc_list.append(acc)
                season_list.append((k+1))
    return u_list, acc_list,season_list,train_lamda,test_lamda

In [378]:
watch = read('train.txt')

In [379]:
u_list, acc_list,season_list,train_lamda,test_lamda = trainSVM(watch)

the result of season 1 and lamda is: 0.01 eta is: 0.0196078431372549
Accuracy is: [0.28, 0.3, 0.44, 0.48, 0.56, 0.6, 0.7, 0.72, 0.74, 0.72, 0.72, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.74, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.74, 0.76, 0.76, 0.78, 0.8, 0.76]
Step is: [30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 360, 390, 420, 450, 480, 510, 540, 570, 600, 630, 660, 690, 720, 750, 780, 810, 840, 870, 900]
[matrix([[ 0.23485308],
        [-0.15955488],
        [ 0.41088687],
        [ 1.29225129],
        [ 0.6190471 ],
        [ 0.46450753]]), -0.9215686274509747]
the result of season 2 and lamda is: 0.01 eta is: 0.019230769230769232
Accuracy is: [0.3, 0.36, 0.4, 0.4, 0.42, 0.48, 0.52, 0.56, 0.58, 0.64, 0.64, 0.68, 0.72, 0.78, 0.8, 0.78, 0.76, 0.78, 0.78, 0.76, 0.76, 0.76, 0.78, 0.78, 0.78, 0.8, 0.8, 0.8, 0.8, 0.8]
Step is: [30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 360, 390, 420, 450, 480, 510, 540, 570, 600, 630, 660, 690, 720, 750, 780, 810, 840, 870, 900]
[matrix([

/Users/xy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':


the result of season 4 and lamda is: 0.01 eta is: 0.018518518518518517
Accuracy is: [0.34, 0.38, 0.4, 0.44, 0.52, 0.56, 0.6, 0.66, 0.7, 0.7, 0.74, 0.78, 0.78, 0.78, 0.88, 0.86, 0.84, 0.82, 0.86, 0.84, 0.9, 0.9, 0.92, 0.94, 0.96, 0.92, 0.94, 0.92, 0.9, 0.92]
Step is: [30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 360, 390, 420, 450, 480, 510, 540, 570, 600, 630, 660, 690, 720, 750, 780, 810, 840, 870, 900]
[matrix([[ 0.27885462],
        [-0.15931086],
        [ 0.36774147],
        [ 1.33772131],
        [ 0.46715539],
        [ 0.47072975]]), -0.8518518518518585]
the result of season 5 and lamda is: 0.01 eta is: 0.01818181818181818
Accuracy is: [0.44, 0.46, 0.5, 0.56, 0.6, 0.62, 0.64, 0.66, 0.66, 0.66, 0.6, 0.6, 0.62, 0.62, 0.68, 0.68, 0.7, 0.72, 0.72, 0.72, 0.72, 0.72, 0.68, 0.68, 0.66, 0.68, 0.66, 0.7, 0.72, 0.7]
Step is: [30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 360, 390, 420, 450, 480, 510, 540, 570, 600, 630, 660, 690, 720, 750, 780, 810, 840, 870, 900]
[matrix([

the result of season 19 and lamda is: 0.01 eta is: 0.014492753623188406
Accuracy is: [0.28, 0.4, 0.4, 0.44, 0.48, 0.5, 0.52, 0.52, 0.52, 0.6, 0.64, 0.66, 0.7, 0.68, 0.7, 0.72, 0.72, 0.72, 0.74, 0.74, 0.74, 0.72, 0.72, 0.74, 0.72, 0.74, 0.74, 0.74, 0.76, 0.76]
Step is: [30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 360, 390, 420, 450, 480, 510, 540, 570, 600, 630, 660, 690, 720, 750, 780, 810, 840, 870, 900]
[matrix([[ 0.37710671],
        [-0.08550551],
        [ 0.59551492],
        [ 1.45599921],
        [ 0.20695876],
        [ 0.28517455]]), -0.9130434782608758]
the result of season 21 and lamda is: 0.01 eta is: 0.014084507042253521
Accuracy is: [0.32, 0.36, 0.38, 0.42, 0.46, 0.52, 0.52, 0.58, 0.62, 0.64, 0.64, 0.66, 0.66, 0.7, 0.74, 0.76, 0.82, 0.82, 0.82, 0.86, 0.86, 0.86, 0.88, 0.88, 0.86, 0.84, 0.84, 0.84, 0.84, 0.84]
Step is: [30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 360, 390, 420, 450, 480, 510, 540, 570, 600, 630, 660, 690, 720, 750, 780, 810, 840, 870, 900]

the result of season 36 and lamda is: 0.01 eta is: 0.011627906976744186
Accuracy is: [0.3, 0.32, 0.34, 0.36, 0.4, 0.42, 0.42, 0.48, 0.52, 0.52, 0.56, 0.58, 0.62, 0.64, 0.66, 0.66, 0.68, 0.72, 0.72, 0.76, 0.76, 0.76, 0.76, 0.82, 0.84, 0.84, 0.84, 0.82, 0.84, 0.86]
Step is: [30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 360, 390, 420, 450, 480, 510, 540, 570, 600, 630, 660, 690, 720, 750, 780, 810, 840, 870, 900]
[matrix([[0.40733014],
        [0.34923189],
        [0.6975705 ],
        [1.35669927],
        [1.16156837],
        [0.56399852]]), -0.7674418604651219]
the result of season 37 and lamda is: 0.01 eta is: 0.011494252873563218
Accuracy is: [0.38, 0.4, 0.4, 0.46, 0.5, 0.52, 0.52, 0.56, 0.56, 0.54, 0.58, 0.56, 0.56, 0.56, 0.58, 0.58, 0.64, 0.64, 0.64, 0.66, 0.7, 0.7, 0.74, 0.74, 0.74, 0.76, 0.78, 0.76, 0.78, 0.76]
Step is: [30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 360, 390, 420, 450, 480, 510, 540, 570, 600, 630, 660, 690, 720, 750, 780, 810, 840, 870, 900]
[matr

the result of season 53 and lamda is: 0.01 eta is: 0.009708737864077669
Accuracy is: [0.34, 0.32, 0.34, 0.38, 0.42, 0.44, 0.44, 0.42, 0.44, 0.48, 0.5, 0.54, 0.54, 0.56, 0.56, 0.58, 0.6, 0.6, 0.62, 0.62, 0.64, 0.66, 0.66, 0.68, 0.66, 0.68, 0.72, 0.72, 0.74, 0.74]
Step is: [30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 360, 390, 420, 450, 480, 510, 540, 570, 600, 630, 660, 690, 720, 750, 780, 810, 840, 870, 900]
[matrix([[0.6752428 ],
        [0.32212185],
        [0.67942755],
        [1.39587844],
        [1.05259997],
        [0.38861429]]), -0.6990291262135907]
the result of season 54 and lamda is: 0.01 eta is: 0.009615384615384616
Accuracy is: [0.3, 0.36, 0.4, 0.46, 0.48, 0.46, 0.46, 0.5, 0.52, 0.52, 0.5, 0.54, 0.56, 0.62, 0.64, 0.64, 0.64, 0.64, 0.66, 0.68, 0.68, 0.7, 0.7, 0.68, 0.7, 0.72, 0.72, 0.72, 0.72, 0.74]
Step is: [30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 360, 390, 420, 450, 480, 510, 540, 570, 600, 630, 660, 690, 720, 750, 780, 810, 840, 870, 900]
[matrix(

In [380]:
def readtest(filename):
    data_raw = pd.read_csv(filename, header = None)
    data_raw.drop([1,3,5,6,7,8,9,13],axis = 1, inplace= True)
    data_raw.columns = [0,1,2,3,4,5]
    return(data_raw)

In [381]:
def getx(data):
    data_mean = np.zeros(6)
    data_std = np.zeros(6)
    p = data[0].count()
    x = np.zeros((p,6))
    for i in range(6):
        data_mean[i] = data[i].mean()#data_var[i] = data[i].var()
        data_std[i] = data[i].std()
    for m in range(p):
        for n in range(6): #x[m][n] = ((data[n][m] - data_mean[n])/math.sqrt(data_var[n])
            x[m][n] = ((data[n][m] - data_mean[n])/(data_std[n]))
    return x 

In [420]:
def write(best_u):
    test = readtest('test.txt')
    x2 = getx(test)
    x2 = np.matrix(x2)
    sign = []
    Sign = ''
    for i in range(len(x2)):
        s = np.dot(u[0].T,x2[i].T)+u[1]
        s = np.sign(s)
        sign.append(s)
    for j in range(len(sign)):
        if sign[j] == -1:
            Sign = Sign+'<=50K\n'
        else:
            Sign = Sign+'>50K\n'
    f = open('submission.txt','w')
    f.write(Sign)
    f.close

In [408]:
def getyita(u_list, acc_list,season_list):
    l = np.zeros(len(acc_list))
    o = np.zeros(len(acc_list))
    for i in range(len(acc_list)):
        count = 0
        f = len(acc_list[i])
        for j in range(int(f/2),f):
            count = count+acc_list[i][j]
        acc = count/f
        l[i] = acc
        o[i] = acc_list[i][f-1]
    max(l)
    m = np.argmax(l)
    yita1 = 1/(season_list[m]+50)
    n = np.argmax(o)
    yita2 = 1/(season_list[n]+50)
    print('season that has best overall accuracy is:',season_list[m], 'season that has best last accuracy is:',season_list[n])
    print('corresponding yitas are yita1:', yita1, 'yita2:',yita2)
    return m,n,yita1,yita2

In [409]:
s,n,yita1,yita2 = getyita(u_list, acc_list,season_list)

season that has best overall accuracy is: 18 season that has best last accuracy is: 4
corresponding yitas are yita1: 0.014705882352941176 yita2: 0.018518518518518517


In [410]:
def getlamda_u(yita,train_lamda,test_lamda):
    u_lam_list = []
    acc_lam_list = []
    lam_list = []
    for lam in [1e-3,1e-2,1e-1,1,10,100]:
        a_est = np.full((6,1),1)
        a_est = np.matrix(a_est)
        b_est = 3
        new_u = [a_est,b_est]
        x, y = getxy(train_lamda)
        eta = yita
        steps = 600
        x = np.matrix(x)
        acc = []
        stepac = []
        new_u_list = []
        x1,y1 = getxy(test_lamda)
        for i in range(steps):
            n = np.random.randint(0, len(x)-1) # Batch size of 1.
            g = gradient_of_cost(x[n].T, y[n], new_u, lam)
            new_u[0] = new_u[0] - eta*g[0]
            new_u[1] = new_u[1] - eta*g[1]
            if (i+1)%30 == 0:
                if accuracy(new_u[0], new_u[1], x1,y1)!=0:
                    acc.append(accuracy(new_u[0], new_u[1], x1,y1))
                    stepac.append((i+1))
                    new_u_list.append(new_u)
        if acc != []:
            print( 'lamda is:', lam,'eta is:',eta)
            print('Accuracy is:',acc)
            print('Step is:',stepac)
            print(new_u)
            u_lam_list.append(new_u_list)
            acc_lam_list.append(acc)
            lam_list.append(lam)
    return u_lam_list,acc_lam_list,lam_list

In [411]:
u_lam_list,acc_lam_list,lam_list = getlamda_u(yita1,train_lamda,test_lamda)

lamda is: 0.001 eta is: 0.014705882352941176
Accuracy is: [0.3584090909090909, 0.3922727272727273, 0.41795454545454547, 0.4547727272727273, 0.49636363636363634, 0.5336363636363637, 0.58, 0.6011363636363637, 0.6193181818181818, 0.6386363636363637, 0.6602272727272728, 0.6861363636363637, 0.7152272727272727, 0.7172727272727273, 0.7297727272727272, 0.7536363636363637, 0.7745454545454545, 0.7770454545454546, 0.7840909090909091, 0.788409090909091]
Step is: [30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 360, 390, 420, 450, 480, 510, 540, 570, 600]
[matrix([[0.75807623],
        [0.09415544],
        [0.84302458],
        [1.443755  ],
        [1.05214625],
        [0.63898999]]), -0.6323529411764615]
lamda is: 0.01 eta is: 0.014705882352941176
Accuracy is: [0.3463636363636364, 0.38727272727272727, 0.4084090909090909, 0.4490909090909091, 0.48772727272727273, 0.5145454545454545, 0.5611363636363637, 0.5893181818181819, 0.6063636363636363, 0.65, 0.6729545454545455, 0.6970454545454545, 0.715

In [412]:
u_lam_list1,acc_lam_list1,lam_list1 = getlamda_u(yita2,train_lamda,test_lamda)

lamda is: 0.001 eta is: 0.018518518518518517
Accuracy is: [0.35704545454545455, 0.39295454545454545, 0.43136363636363634, 0.44431818181818183, 0.48727272727272725, 0.5545454545454546, 0.6025, 0.6363636363636364, 0.6820454545454545, 0.6990909090909091, 0.7056818181818182, 0.7254545454545455, 0.7365909090909091, 0.7625, 0.7838636363636363, 0.7895454545454546, 0.7929545454545455, 0.8043181818181818, 0.8045454545454546, 0.8111363636363637]
Step is: [30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 360, 390, 420, 450, 480, 510, 540, 570, 600]
[matrix([[0.30077263],
        [0.21181772],
        [0.70473454],
        [1.48142649],
        [0.67020613],
        [0.44455043]]), -0.907407407407414]
lamda is: 0.01 eta is: 0.018518518518518517
Accuracy is: [0.3679545454545455, 0.4143181818181818, 0.4531818181818182, 0.5025, 0.5452272727272728, 0.5938636363636364, 0.6281818181818182, 0.6677272727272727, 0.7059090909090909, 0.7252272727272727, 0.7504545454545455, 0.7786363636363637, 0.7838636363

In [422]:
def getbestlam_u(u_lam_list,acc_lam_list,lam_list,u_lam_list1,acc_lam_list1,lam_list1):
    acc_array = np.asarray(acc_lam_list)
    acc_array1 = np.asarray(acc_lam_list1)
    j,k = acc_array.shape
    j1,k1 = acc_array1.shape
    max_i = np.zeros(j)
    max_i1 = np.zeros(j1)
    l = np.zeros(j)
    l1 = np.zeros(j1)
    for i in range(j):
        max_i[i] = np.max(acc_array[i])
        l[i] = np.argmax(acc_array[i])
    for t in range(j1):
        max_i1[i] = np.max(acc_array1[i])
        l1[i] = np.argmax(acc_array1[i])
    n = np.max(max_i)   
    m = np.argmax(max_i)
    p = l[m]
    n1 = np.max(max_i1)   
    m1 = np.argmax(max_i1)
    p1 = l1[m1]
    if max(n,n1) == n:
        best_lam = lam_list[m]
        best_u = u_lam_list[int(m)][int(p)]
        print('best yita is yita1','m is', m ,'p is', p)
    if max(n,n1) == n1:
        best_lam = lam_list1[m1]
        best_u = u_lam_list1[int(m1)][int(p1)]
        print('best yita is yita2','m is', m ,'p is', p)
    return best_lam, best_u

In [423]:
best_lam, best_u = getbestlam_u(u_lam_list,acc_lam_list,lam_list,u_lam_list1,acc_lam_list1,lam_list1)

best yita is yita1 m is 2 p is 18.0


In [424]:
best_lam

0.1

In [425]:
best_u

[matrix([[0.24776708],
         [0.13061276],
         [0.48940212],
         [0.73737312],
         [0.29176335],
         [0.30933258]]), -0.9852941176470484]

In [426]:
write(best_u)